In [2]:
# Uncomment the following lines if you are using Windows!
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

spark = (
    SparkSession.builder
    .appName("Big Data Analytics")
    .master("local")
    .config("spark.driver.host", '127.0.0.1')
    .getOrCreate()
)

## Ingest data from CSV and write to database

In [6]:
from pyspark.sql.functions import *
# Read in CSV files
fifa_df = {}
for i in range(15, 23):
   fifa_df[i] = (spark.read.csv("data/players_{}.csv".format(i),header=True, inferSchema= True))


In [7]:
# Add year column to each dataframe
for key in fifa_df.keys():
    fifa_df[key] = fifa_df[key].withColumn("year", lit((2000 + key)))

+---------+--------------------+----------+--------------------+----------------+-------+---------+---------+--------+---+----------+---------+---------+------------+------------+--------------------+------------+-------------+------------------+----------------+-----------+-------------------------+--------------+----------------+--------------+---------------+--------------------+--------------+---------+-----------+------------------------+----------+-------------+---------+------------------+--------------------+--------------------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+---------------------

In [10]:

from functools import reduce
# Merge all dataframes into one
complete_df = reduce(DataFrame.union, [fifa_df[key] for key in fifa_df])

In [11]:
complete_df.show(1,vertical = True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [12]:
# Assign unique Id to each row
complete_df = complete_df.withColumn('UID', monotonically_increasing_id())

## Prepare truncated data for cloud usage

In [21]:
# complete_df = complete_df.sample(False, 0.1)

In [22]:
# pandas_df = complete_df.toPandas()

# pandas_df.to_csv('merged_raw_data.csv', header=True, index = False)

## Write to Postgres database

In [23]:

# Ingest data into Postgres
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db passwor
db_properties['password']= "1234"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "fifa.players"

# Uncomment this if you haven't written to Postgres
# complete_df.write.format("jdbc")\
# .mode("overwrite")\
# .option("url", db_properties['url'])\
# .option("dbtable", db_properties['table'])\
# .option("user", db_properties['username'])\
# .option("password", db_properties['password'])\
# .option("Driver", db_properties['driver'])\
# .option("jar", "C:/Users/jimmy/spark-3.4.1-bin-hadoop3/jars/postgresql-42.6.0.jar") \
# .save()

In [24]:
df_read = spark.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [25]:
df_read.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 

## Query 1: Top club_num clubs have the highest number of players with contracts ending in 2022

In [26]:
# Analytical Queries Section
from pyspark.sql import functions as F

club_num = input("Enter the number of clubs you want to analyze: ")
print("You entered: " + club_num)


df_2022 = df_read.filter((df_read['Year']== 2022) & (df_read["club_contract_valid_until"] == 2023))

club_counts = df_2022.groupBy("club_name").count().orderBy(F.desc("count"))
club_counts.show(int(club_num))


You entered: 5
+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|En Avant de Guingamp|   19|
| Club Atlético Lanús|   17|
|       Lechia Gdańsk|   17|
|            Barnsley|   16|
|        Kasimpaşa SK|   16|
+--------------------+-----+
only showing top 5 rows



## Query 2  count the number of players older than 27 years old
## for each club every year, calculate the averages and list the Y clubs with highest averages

In [27]:
# Check if the club_name is null and age is greater than 27
older_players = df_read.filter((df_read["age"] > 27) & (df_read["club_name"].isNotNull()))

# Group by club_name and Year
club_year_counts = older_players.groupBy("club_name", "Year").agg(F.count("UID").alias("total_count"))

# Group by club_name to calculate average
club_avg_counts = club_year_counts.groupBy("club_name").agg(F.avg("total_count").alias("average_count"))

# Order by average in descending order
sorted_clubs = club_avg_counts.orderBy(F.desc("average_count"))

# Display the top Y clubs, handling tie scenarios
num_club = int(input("Enter the number of top clubs you want to see: "))
print("You entered: " + str(num_club))

threshold = sorted_clubs.limit(num_club).select("average_count").collect()[num_club-1]["average_count"]
    
filtered_result = sorted_clubs.filter(sorted_clubs["average_count"] >= threshold).show()

# Result
filtered_result


You entered: 10
+--------------------+-------------+
|           club_name|average_count|
+--------------------+-------------+
|  Dorados de Sinaloa|         19.0|
| Matsumoto Yamaga FC|         19.0|
| Shanghai Shenhua FC|         18.5|
|          Qingdao FC|         18.0|
|Club Deportivo Jo...|         17.5|
|            Altay SK|         17.0|
|         Guaireña FC|         17.0|
|İstanbul Başakşeh...|       16.625|
|      BB Erzurumspor|         16.5|
|        Club Olimpia|         16.5|
|      Sport Huancayo|         16.5|
+--------------------+-------------+



## Query 3 What is the most frequent nation_position in the dataset for each year?

In [28]:
# Group by Year and nation_position
df_read_filter = df_read.filter(df_read["nation_position"].isNotNull())
nation_position_counts = df_read_filter.groupBy("Year", "nation_position").agg(F.count("UID").alias("total_count"))

# Find the max count for each year
max_count = nation_position_counts.groupBy("Year").agg(F.max("total_count").alias("max_count"))
max_count = max_count.withColumnRenamed("Year", "Year2")

# Join the two table to find which position has the max count for each year
top_nation_positions = nation_position_counts.join(max_count, (nation_position_counts["Year"] == max_count["Year2"]) & 
                                                   (nation_position_counts["total_count"] == max_count["max_count"]))

# Result
top_nation_positions.select("Year", "nation_position", "max_count").orderBy(F.desc("Year")).show()

+----+---------------+---------+
|Year|nation_position|max_count|
+----+---------------+---------+
|2022|            SUB|      396|
|2021|            SUB|      588|
|2020|            SUB|      588|
|2019|            SUB|      576|
|2018|            SUB|      600|
|2017|            SUB|      564|
|2016|            SUB|      511|
|2015|            SUB|      564|
+----+---------------+---------+

